In [ ]:
#import gm as gm
import gwb as gwb
from gwb import GM as gm
from wpca import WPCA


import numpy as np
import ot
import obj
import trimesh
from tqdm import tqdm

import pl
import matplotlib.cm as cm
#import plotly.graph_objects as go
#from plotly.subplots import make_subplots

import open3d as o3d
from keypoint_guided_optimal_transport.keypoint_guided_OT import KeyPointGuidedOT

In [ ]:
def invert_permutation(p):
    """Return an array s with which np.array_equal(arr[p][s], arr) is True.
    The array_like argument p must be some permutation of 0, 1, ..., len(p)-1.
    """
    p = np.asanyarray(p) # in case p is a tuple, etc.
    s = np.empty_like(p)
    s[p] = np.arange(p.size)
    return s

def interpolate_two(X,Y,idxs,meas,P,N_interpolation,paint_init_left=True):
    #colour X and Y according to P
    #cX = (X.g[np.argmin(X.X[:,1])])**4 #+ 1/2 * 
    cX = np.linalg.norm(X.X - X.X[np.argmin(np.linalg.norm(X.X,axis=1))],axis=1)
    cY = (P.T / np.sum(P,axis=1)).dot(cX)

    #interpolate
    #weights
    ws = np.flip((np.stack([np.arange(1,N_interpolation+1,1),np.flip(np.arange(1,N_interpolation+1,1))])/(N_interpolation+1)).T,axis=0)

    #embedding for barycenters
    bary_embs = []
    bary_embs_3d = []
    bary_tris = gwb.generate_triangles_from_single(Y,idxs,1)
    for w in ws:
        bary_emb = gwb.create_surface_embedding([X,Y],idxs,meas,ws=np.sqrt(w))
        bary_embs.append(bary_emb)

        #WPCA for dimensionality reduction
        PCA = WPCA(n_components=3)
        bary_emb_3d = PCA.fit_transform(bary_emb,**{'weights': [[b]*6 for b in meas]})
        bary_embs_3d.append(bary_emb_3d)

    #colour barycenters
    bary_c = [w[0]*cX[idxs[:,0]] + w[1] * cY[idxs[:,1]] for w in ws]
    
    return cX,cY,bary_embs,bary_embs_3d,bary_c

def make_o3dmesh_from_node_tri_col(X,tri,col):
    mesh = o3d.geometry.TriangleMesh()
    mesh.vertices = o3d.utility.Vector3dVector(X)
    mesh.triangles = o3d.utility.Vector3iVector(tri)
    mesh.vertex_colors = o3d.utility.Vector3dVector(col)
    return mesh

In [ ]:
sqd = 12000
#sqd = 1000

In [ ]:
#LOAD FAUST
Nodes_FAUST = []
Tris_FAUST = []
for i in range(100):
    filepath= "./MPI-FAUST/training/registrations/tr_reg_{0}.ply".format(str("{:03d}".format(i)))
    pcd = o3d.io.read_triangle_mesh(filepath)
    mesh = trimesh.Trimesh(vertices=pcd.vertices,faces = pcd.triangles)
    #mesh = mesh.simplify_quadratic_decimation(sqd)
    Nodes_FAUST.append(np.array(mesh.vertices))
    Tris_FAUST.append(np.array(mesh.faces))

Nodes_FAUST = np.array(Nodes_FAUST)
Tris_FAUST = np.array(Tris_FAUST)

In [ ]:
#LOAD DEFORM
class_names_DF = ["camel","cat","elephant","face","head","horse","lion"]
lengths_DF = [11,10,11,10,10,11,10]
lengths_by_class_DF = dict(zip(class_names_DF,[+ np.arange(lengths_DF[i]) for i in range(len(lengths_DF))]))
idxs_by_class_DF = dict(zip(class_names_DF,[int(np.sum(lengths_DF[:i])) + np.arange(lengths_DF[i]) for i in range(len(lengths_DF))]))

#LOAD FAUST
Nodes_DF = []
Tris_DF = []
for class_name in class_names_DF:
    for i in lengths_by_class_DF[class_name]:
        filepath = "./DEFORM/{0}-poses/{0}-{1}.obj".format(class_name,str(i).zfill(2))
        #filepath= "./MPI-FAUST/training/registrations/tr_reg_{0}.ply".format(str("{:03d}".format(i)))
        pcd = o3d.io.read_triangle_mesh(filepath)
        mesh = trimesh.Trimesh(vertices=pcd.vertices,faces = pcd.triangles)
        mesh = mesh.simplify_quadratic_decimation(sqd)
        
        Nodes_DF.append(np.array(mesh.vertices))
        Tris_DF.append(np.array(mesh.faces))

In [ ]:
#FAUST: 
#67, 68 als beispiel wo PCA schief gehen kann
#27, 29 opa full mesh?
#82, 92?

In [ ]:
i = 27
j = 29
X = gm(X=Nodes_FAUST[i],Tris=Tris_FAUST[i],mode="surface",gauge_mode="djikstra",squared=False)#,xi="surface_uniform")
Y = gm(X=Nodes_FAUST[j],Tris=Tris_FAUST[j],mode="surface",gauge_mode="djikstra",squared=False)#,xi="surface_uniform")
print("GM spaces generated!")

#One iteration of TB
idxs,meas,Ps = gwb.tb(0,[X,Y],numItermaxEmd=500000)
print("Barycenter computed!")

#Create Embeddings etc
N_interpolation = 5
cX,cY,bary_embs,bary_embs_3d,bary_c = interpolate_two(X,Y,idxs,meas,Ps[1],N_interpolation)
bary_tris = gwb.generate_triangles_from_single(X,idxs,0)
print("Surfaces constructed!")

#save meshes
tmp = cX / np.max(cX)
col = cm.viridis(tmp)[:,:3]
mesh = make_o3dmesh_from_node_tri_col(X.X,X.Tris,col)
o3d.io.write_triangle_mesh(filename="./3d/FAUST_{0}_{1}_X.ply".format(i,j),mesh=mesh)

tmp = cY / np.max(cY)
col = cm.viridis(tmp)[:,:3]
mesh = make_o3dmesh_from_node_tri_col(Y.X,Y.Tris,col)
o3d.io.write_triangle_mesh(filename="./3d/FAUST_{0}_{1}_Y.ply".format(i,j),mesh=mesh)

for k in range(N_interpolation):
    tmp = bary_c[k] / np.max(bary_c[k])
    col = cm.viridis(tmp)[:,:3]
    mesh = make_o3dmesh_from_node_tri_col(bary_embs_3d[k],bary_tris,col)
    o3d.io.write_triangle_mesh(filename="./3d/FAUST_{0}_{1}_B_{2}.ply".format(i,j,k),mesh=mesh)
    
print("Outputs saved!")

In [ ]:
idxs_FAUST = [[27,29],[35,36],[44,54],[53,63]]
for i,j in idxs_FAUST:
    X = gm(X=Nodes_FAUST[i],Tris=Tris_FAUST[i],mode="surface",gauge_mode="djikstra",squared=False)#,xi="surface_uniform")
    Y = gm(X=Nodes_FAUST[j],Tris=Tris_FAUST[j],mode="surface",gauge_mode="djikstra",squared=False)#,xi="surface_uniform")
    print("GM spaces generated!")

    #One iteration of TB
    idxs,meas,Ps = gwb.tb(0,[X,Y],numItermaxEmd=500000)
    print("Barycenter computed!")

    #Create Embeddings etc
    N_interpolation = 5
    cX,cY,bary_embs,bary_embs_3d,bary_c = interpolate_two(X,Y,idxs,meas,Ps[1],N_interpolation)
    bary_tris = gwb.generate_triangles_from_single(X,idxs,0)
    print("Surfaces constructed!")

    #save meshes
    tmp = cX / np.max(cX)
    col = cm.viridis(tmp)[:,:3]
    mesh = make_o3dmesh_from_node_tri_col(X.X,X.Tris,col)
    o3d.io.write_triangle_mesh(filename="./3d/FAUST_{0}_{1}_X.ply".format(i,j),mesh=mesh)

    tmp = cY / np.max(cY)
    col = cm.viridis(tmp)[:,:3]
    mesh = make_o3dmesh_from_node_tri_col(Y.X,Y.Tris,col)
    o3d.io.write_triangle_mesh(filename="./3d/FAUST_{0}_{1}_Y.ply".format(i,j),mesh=mesh)

    for k in range(N_interpolation):
        tmp = bary_c[k] / np.max(bary_c[k])
        col = cm.viridis(tmp)[:,:3]
        mesh = make_o3dmesh_from_node_tri_col(bary_embs_3d[k],bary_tris,col)
        o3d.io.write_triangle_mesh(filename="./3d/FAUST_{0}_{1}_B_{2}.ply".format(i,j,k),mesh=mesh)
    print("Outputs saved!")

In [ ]:
idxs_DF = [[idxs_by_class_DF["lion"][-1], idxs_by_class_DF["lion"][-2]],
            [idxs_by_class_DF["horse"][3], idxs_by_class_DF["camel"][3]]
          ]
for i,j in idxs_DF:
    X = gm(X=Nodes_DF[i],Tris=Tris_DF[i],mode="surface",gauge_mode="djikstra",squared=False)#,xi="surface_uniform")
    Y = gm(X=Nodes_DF[j],Tris=Tris_DF[j],mode="surface",gauge_mode="djikstra",squared=False)#,xi="surface_uniform")
    print("GM spaces generated!")

    #One iteration of TB
    idxs,meas,Ps = gwb.tb(0,[X,Y],numItermaxEmd=500000)
    print("Barycenter computed!")
    print(len(idxs))
    #Create Embeddings etc
    N_interpolation = 5
    cX,cY,bary_embs,bary_embs_3d,bary_c = interpolate_two(X,Y,idxs,meas,Ps[1],N_interpolation)
    bary_tris = gwb.generate_triangles_from_single(X,idxs,0)
    print("Surfaces constructed!")

    #save meshes
    tmp = cX / np.max(cX)
    col = cm.viridis(tmp)[:,:3]
    mesh = make_o3dmesh_from_node_tri_col(X.X,X.Tris,col)
    o3d.io.write_triangle_mesh(filename="./3d/DF_{0}_{1}_X.ply".format(i,j),mesh=mesh)

    tmp = cY / np.max(cY)
    col = cm.viridis(tmp)[:,:3]
    mesh = make_o3dmesh_from_node_tri_col(Y.X,Y.Tris,col)
    o3d.io.write_triangle_mesh(filename="./3d/DF_{0}_{1}_Y.ply".format(i,j),mesh=mesh)

    for k in range(N_interpolation):
        tmp = bary_c[k] / np.max(bary_c[k])
        col = cm.viridis(tmp)[:,:3]
        mesh = make_o3dmesh_from_node_tri_col(bary_embs_3d[k],bary_tris,col)
        o3d.io.write_triangle_mesh(filename="./3d/DF_{0}_{1}_B_{2}.ply".format(i,j,k),mesh=mesh)
    print("Outputs saved!")